# TMNet

In [1]:
import os
import pathlib
import tensorflow as tf
import pandas as pd
import wandb

from bcd.model.network.base import NetworkConfig
from bcd.model.network.tmnet import TMNetConfig, TMNetFactory
from bcd.model.repo import ExperimentRepo
from bcd.model.base import *
from bcd.model.experiment import FeatureExtractionExperiment
from bcd.model.config import *

pd.set_option('display.max_rows',999)

## Configuration

In [2]:
# Experiment Parameters
%env "WANDB_NOTEBOOK_NAME" "tmnet.ipynb"
mode = "Production"
force = False  # Whether to retrain if the model and config already exists.
base_models = [DenseNet(), EfficientNet(), Inception(), InceptionResNet(), MobileNet(), ResNet(), Xception()]

env: "WANDB_NOTEBOOK_NAME"="tmnet.ipynb"


In [3]:
def create_config(network_config: NetworkConfig):
    project_config = ProjectConfig(mode=mode)

    train_config = TrainConfig(epochs=50, learning_rate=1e-4)    

    dataset_config = DatasetConfig(mode=mode)

    checkpoint_config = CheckPointConfig(monitor="val_accuracy", verbose=1, save_best_only=True, save_weights_only=False, mode="auto")

    early_stop_config = EarlyStopConfig(min_delta=1e-4, monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)

    learning_rate_schedule_config = LearningRateScheduleConfig(min_delta=1e-4, monitor="val_loss", factor=0.5, patience=3, restore_best_weights=True, verbose=1, mode="auto")

    config = Config(project=project_config, 
                    dataset=dataset_config, 
                    train=train_config, 
                    network=network_config, 
                    checkpoint=checkpoint_config, 
                    early_stop=early_stop_config, 
                    learning_rate_schedule=learning_rate_schedule_config)
    return config

network_config = TMNetConfig(activation="sigmoid", input_shape=(224,224,3), output_shape=1, dense1=1024, dense2=1024)
config = create_config(network_config=network_config)

## Load Data

In [4]:
train_dir = pathlib.Path(config.dataset.train_dir).with_suffix('') 
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

# Validation DataSet (10%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

Found 2471 files belonging to 2 classes.
Using 1977 files for training.
Found 2471 files belonging to 2 classes.
Using 494 files for validation.


## Callbacks

In [5]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=config.early_stop.monitor, 
                                                       min_delta=config.early_stop.min_delta,
                                                       patience=config.early_stop.patience, 
                                                       restore_best_weights=config.early_stop.restore_best_weights,
                                                       verbose=config.early_stop.verbose)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=config.learning_rate_schedule.monitor,
                                                          factor=config.learning_rate_schedule.factor,
                                                          patience=config.learning_rate_schedule.patience,
                                                          verbose=config.learning_rate_schedule.verbose,
                                                          mode=config.learning_rate_schedule.mode,
                                                          min_delta=config.learning_rate_schedule.min_delta,
                                                          min_lr=config.learning_rate_schedule.min_lr)
callbacks = [early_stop_callback, reduce_lr_callback]

## Dependencies

In [6]:
repo = ExperimentRepo(mode = mode, project=config.project.name)
optimizer=tf.keras.optimizers.Adam
metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

## Build Factory

In [7]:
factory = TMNetFactory(config=config.network)

## Run Experiments

In [8]:
for base_model in base_models:
    network = factory.create(base_model=base_model)
    experiment = FeatureExtractionExperiment(network=network, config=config, optimizer=optimizer, repo=repo, callbacks=callbacks, metrics=metrics, tags=[network.architecture, base_model.name], force=False)
    experiment.run(train_ds=train_ds, val_ds=val_ds)


# ================================================================================================ #
                                           TMNet_DenseNet                                           
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 TMNet_DenseNet_input_layer   [(None, 224, 224, 3)]    0         
 (InputLayer)                                                    
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 tf.math.truediv_1 (TF

wandb: Currently logged in as: aistudio. Use `wandb login --relogin` to force relogin


Epoch 1/50
31/31 [==============================] - ETA: 0s - loss: 0.6675 - accuracy: 0.5943 - auc: 0.6247 - precision: 0.5585 - recall: 0.4719
Epoch 1: val_accuracy improved from -inf to 0.61943, saving model to models/production/TMNet_DenseNet-8wuoxn2a.keras
31/31 [==============================] - 419s 13s/step - loss: 0.6675 - accuracy: 0.5943 - auc: 0.6247 - precision: 0.5585 - recall: 0.4719 - val_loss: 0.6702 - val_accuracy: 0.6194 - val_auc: 0.6738 - val_precision: 0.5493 - val_recall: 0.8326 - lr: 1.0000e-04
Epoch 2/50
31/31 [==============================] - ETA: 0s - loss: 0.5573 - accuracy: 0.7172 - auc: 0.7900 - precision: 0.7051 - recall: 0.6393
Epoch 2: val_accuracy improved from 0.61943 to 0.62753, saving model to models/production/TMNet_DenseNet-8wuoxn2a.keras
31/31 [==============================] - 389s 12s/step - loss: 0.5573 - accuracy: 0.7172 - auc: 0.7900 - precision: 0.7051 - recall: 0.6393 - val_loss: 0.6562 - val_accuracy: 0.6275 - val_auc: 0.6715 - val_preci

wandb: Waiting for artifact run-8wuoxn2a-TMNet_DenseNet-8wuoxn2a.keras to be committed...



wandb: Committed artifact aistudio/Breast-Cancer-Detection-Production/run-8wuoxn2a-TMNet_DenseNet-8wuoxn2a.keras:v0
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

# ================================================================================================ #
                                           TMNet_Xception                                           
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 TMNet_Xception_input_layer   [(None, 224, 224, 3)]    0         
 (InputLayer)                                                    
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                      

Epoch 1/50
31/31 [==============================] - ETA: 0s - loss: 0.7116 - accuracy: 0.5579 - auc: 0.5860 - precision: 0.5245 - recall: 0.4725
Epoch 1: val_accuracy did not improve from 0.66802

Epoch 1: val_accuracy did not improve from 0.66802

Epoch 1: val_accuracy did not improve from 0.66802

Epoch 1: val_accuracy did not improve from 0.66802

Epoch 1: val_accuracy did not improve from 0.66802

Epoch 1: val_accuracy did not improve from 0.63968

Epoch 1: val_accuracy improved from -inf to 0.56275, saving model to models/production/TMNet_Xception-70fsj1k8.keras
31/31 [==============================] - 218s 7s/step - loss: 0.7116 - accuracy: 0.5579 - auc: 0.5860 - precision: 0.5245 - recall: 0.4725 - val_loss: 0.6840 - val_accuracy: 0.5628 - val_auc: 0.5646 - val_precision: 0.5155 - val_recall: 0.3756 - lr: 1.0000e-04
Epoch 2/50
31/31 [==============================] - ETA: 0s - loss: 0.5897 - accuracy: 0.6758 - auc: 0.7486 - precision: 0.6761 - recall: 0.5371
Epoch 2: val_accurac

wandb: Waiting for artifact run-70fsj1k8-TMNet_Xception-70fsj1k8.keras to be committed...



wandb: Committed artifact aistudio/Breast-Cancer-Detection-Production/run-70fsj1k8-TMNet_Xception-70fsj1k8.keras:v0
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch/accuracy,▁▁▁▃▃▄▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████████████
epoch/auc,▁▁▁▄▄▅▅▆▆▇▇▇▇▇▇▇▇▇██████████████████████
epoch/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
epoch/learning_rate,██████████████████████▄▄▄▄▄▄▄▂▂▂▂▂▂▁▁▁▁▁
epoch/loss,███▇▇▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/lr,█████████████████████████▄▄▄▄▄▄▂▂▂▂▂▂▂▁▁
epoch/precision,▁▁▁▃▃▅▅▅▅▆▆▆▆▆▇▇▆▆▇▇▇▇▇▇▇███████████████
epoch/recall,▁▁▁▂▂▄▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████████████
epoch/val_accuracy,▁▁▁▂▂▅▅▃▃▂▂▅▅▅▆▆▆▆▇▇▄▄▆▆▆▆▆▇▇▅▅▅▅██▆▆▆██
epoch/val_auc,▁▁▁▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▆▆▇▇▇▇▇██▇▇█████████
epoch/val_loss,▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▄▄▄▃▃▄▄▆▆▆▆▇▇█████
